# Permutation of source time courses for testing

In [1]:
import os
from pathlib import Path
import sys
import mne 
import numpy as np
from mne.datasets import fetch_fsaverage
from tabulate import tabulate
import scipy.io as sio
import csv
import random

fs_dir = fetch_fsaverage(verbose=True)
subject = "fsaverage"
subjects_dir = os.path.dirname(fs_dir)
#will need to change the path below
sys.path.append("C:/Users/EISERK01/eeg-projects/Code/eeg_toolkit")
from eeg_toolkit import preprocess, utils
import eeg_toolkit.functional_connectivity as fc
import eeg_toolkit.permutation_testing as pt

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage
0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

#preprocessing method
sl_method = 'MNE'

# data paths
data_dir = Path("../../Data")
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

[-2.5,0.0,2.5]


### Get subset of subjects with and without widespread pain

### Manually enter age/sex matched subjects

In [3]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'  
]
  
sub_ids = sub_ids_CP + sub_ids_HC

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

### Low Back Pain Laterality

In [4]:
left_pain_ids = [
    '016',
    '017',
    '018',
    '019',
    '020',
    '023',
    '024',
    '025',
    '027',
    '028',
    '029',
    '030',
    '032',
    '033',
    '036',
    '038',
    '045',
    '052',
    '054',
    '055',
    '057',
    '058',
]

right_pain_ids = [
    '021',
    '026',
    '031',
    '037',
    '039',
    '043',
    '044',
    '046',
    '047',
    '048',
    '049',
    '050',
    '051',
    '053',
    '056',
]

bilateral_pain_ids = [
    '022',
    '034',
    '035',
    '040',
    '041',
    '042',
    '059',
]

# Randomly assign healthy subjects to left/right/bilateral pain
lst = sub_ids_HC
random.seed(42)
left_pain_ids.extend(random.sample(lst, 8))
lst = [i for i in lst if i not in left_pain_ids]  

# Randomly assign 8 elements to right and remove them from lst  
right_pain_ids.extend(random.sample(lst, 8))
lst = [i for i in lst if i not in right_pain_ids]  
  
# Assign the remaining elements to bilateral  
bilateral_pain_ids.extend(lst)  

In [5]:
for sub_ids, group in zip([sub_ids_CP, sub_ids_WSP, sub_ids_LP, sub_ids_HC],"CP WSP LP HC".split()):
    print(f"\n{group}")
    for sub_pain_ids, laterality in zip([left_pain_ids, right_pain_ids, bilateral_pain_ids],"Left Right Bilateral".split()):
        print(f"{laterality} - {len(set(sub_ids) & set(sub_pain_ids))}")
        print(set(sub_ids) & set(sub_pain_ids))


CP
Left - 8
{'033', '036', '057', '018', '020', '038', '032', '027'}
Right - 8
{'044', '043', '037', '050', '031', '021', '048', '049'}
Bilateral - 3
{'059', '040', '035'}

WSP
Left - 6
{'036', '052', '024', '018', '032', '045'}
Right - 3
{'031', '039', '046'}
Bilateral - 3
{'034', '040', '022'}

LP
Left - 3
{'029', '023', '020'}
Right - 7
{'056', '050', '037', '031', '021', '048', '049'}
Bilateral - 2
{'042', '041'}

HC
Left - 8
{'C10', 'C17', 'C11', 'C18', 'C13', 'C12', 'C9.', 'C24'}
Right - 8
{'C3.', 'C26', 'C7.', 'C14', 'C27', 'C6.', 'C25', 'C15'}
Bilateral - 3
{'C19', 'C16', 'C2.'}


In [6]:
# Exclude HEALTHY subjects with widespread pain
widespread_pain_HC = ['C1.','C23']
sub_ids_HC = [sub for sub in sub_ids_HC if sub not in widespread_pain_HC]

In [7]:
print(f"Chronics (all): {len(sub_ids_CP)}")
print([el for el in sub_ids_CP])
print(f"Chronics with widespread pain: {len(sub_ids_WSP)}")
print([el for el in sub_ids_WSP])
print(f"Chronics with localized pain: {len(sub_ids_LP)}")
print([sub for sub in sub_ids_LP])
print(f"Controls: {len(sub_ids_HC)}")
print([el for el in sub_ids_HC])
print(f"\nTotal (Chronics + Controls): {len(sub_ids)}")

Chronics (all): 19
['018', '020', '021', '027', '031', '032', '033', '035', '036', '037', '038', '040', '043', '044', '048', '049', '050', '057', '059']
Chronics with widespread pain: 12
['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052']
Chronics with localized pain: 12
['020', '021', '023', '029', '031', '037', '041', '042', '048', '049', '050', '056']
Controls: 19
['C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']

Total (Chronics + Controls): 19


### Permute source space time signal, repeat 100

In [8]:
this_sub_ids = sub_ids_CP

In [9]:
repetitions = 1 #takes forever, switch to a smaller number for testing

In [10]:
for j in range(0, repetitions):
# Loop over each group
    for this_sub_ids in [
        sub_ids_CP, 
        sub_ids_WSP, 
        sub_ids_LP, 
        sub_ids_HC,
        ]:
        for sub_id in this_sub_ids:
            count = 1

            # Load the original z-scored epochs data, eyes open data, eyes closed data
            label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
            eo_data = utils.unpickle_data(EO_resting_data_path, f"{sub_id}_eyes_open.pkl")
            ec_data = utils.unpickle_data(EC_resting_data_path, f"{sub_id}_eyes_closed.pkl")

            # Permute the signals one epoch at a time
            for i,el in enumerate(label_ts):
                #print(f"Epoch {i}:",el.shape)
                
                # Create the surrogate data for zscored_epochs
                surrogate_data = pt.create_surrogate_data(el)
                
                # Replace the original data with the surrogate data
                label_ts[i] = surrogate_data

            # Create the surrogate data for eyes open, replace original data with surrogate
            surrogate_data = pt.create_surrogate_data(eo_data)
            eo_data = surrogate_data

            # Create the surrogate data for eyes closed, replace original data with surrogate
            surrogate_data = pt.create_surrogate_data(ec_data)
            ec_data = surrogate_data    
            
            # Save the surrogate data
            subset = "FFT Permuted Data " + str(j+1)
            #make zcored_epochs save path
            fc_subset_path = stc_path / subset / "zscored_Epochs" / time_win_path
            save_path = fc_subset_path
            os.makedirs(save_path, exist_ok=True)
            utils.pickle_data(save_path, f"{sub_id}_epochs.pkl", label_ts)

            #make eyes open save path
            fc_subset_path = stc_path / subset / "Eyes Open"
            save_path = fc_subset_path
            os.makedirs(save_path, exist_ok=True)
            utils.pickle_data(save_path, f"{sub_id}_eyes_open.pkl", eo_data)

            #make eyes closed save path
            fc_subset_path = stc_path / subset / "Eyes Closed"
            save_path = fc_subset_path
            os.makedirs(save_path, exist_ok=True)
            utils.pickle_data(save_path, f"{sub_id}_eyes_closed.pkl", ec_data)

            # Clear display
            utils.clear_display()
            # break

Saved C9._epochs.pkl to ../../Data/Source Time Courses (MNE)/FFT Permuted Data 1/zscored_Epochs/5_sec_time_window.
Saved C9._eyes_open.pkl to ../../Data/Source Time Courses (MNE)/FFT Permuted Data 1/Eyes Open.
Saved C9._eyes_closed.pkl to ../../Data/Source Time Courses (MNE)/FFT Permuted Data 1/Eyes Closed.


### Settings for conditions/bands/methods

In [11]:
# Settings

# Include intermediate stimulus?
include_LS = False

# Data properties
sfreq = 400  # Hz
tmin, tmax = 0.0, 1.25  # exclude the baseline period for connectivity estimation

# Choose which connectivity estimates to run. "amplitude", "phase", or "both"
# method_choice = "phase"
method_choice = "both"

# Load previously saved group data
# load_group_data_flag = True
load_group_data_flag = False

# Save group data?
save_group_data_flag = False
# save_group_data_flag = True
   
# Evoked only or include resting too?
# include_resting = False
include_resting = True


### Define ROIs, frequency bands, conditions, and methods for FC

In [12]:
####################################################################
# REGIONS OF INTEREST
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

# Write out ROI names as acronyms
roi_acronyms = ["rACC-lh", "dACC-lh", "S1-lh", "insula-lh", "dlPFC-lh", "mOFC-lh",
                "rACC-rh", "dACC-rh", "S1-rh", "insula-rh", "dlPFC-rh", "mOFC-rh"]


####################################################################
# BANDS OF INTEREST
Freq_Bands = {
    'delta': [0, 4],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta": [13.0, 30.0],
    "low-gamma": [30.0, 58.5],
    # "notch": [58.5, 61.5],
    "high-gamma": [61.5, 100.0],
}

band_names = [band for band in Freq_Bands]

####################################################################
# CONNECTIVITY METHODS
if method_choice == "phase":
    con_methods = ["wpli2_debiased",
                ]
elif method_choice == "amplitude":
    con_methods = [
        # "aec_pairwise",
        "aec_symmetric",
        ]
elif method_choice == "both":
    con_methods = [
    "wpli2_debiased",
    # "aec_pairwise",
    "aec_symmetric", # keep only symmetric for now
    ]
    
####################################################################
# CONDITIONS
conditions = (
    [
        "Hand 32 mN",
        "Hand 128 mN",
        "Hand 256 mN",
        "Back 32 mN",
        "Back 128 mN",
        "Back 256 mN",
        "Eyes Open",
        "Eyes Closed",
    ]
    if include_LS
    else [
        # "Hand 32 mN",
        # 'Hand LS',
        "Hand 256 mN",
        # "Back 32 mN",
        # 'Back LS',
        # "Back 256 mN",
        "Eyes Open",
        # "Eyes Closed",
    ]
)

# Choose to exclude resting state data
conditions = conditions if include_resting else [
    c for c in conditions if "Eyes" not in c
]

####################################################################
# CONDITION DICTIONARY
condition_dict = {
        "Hand 32 mN": 0,
        'Hand LS': 1,
        "Hand 256 mN": 2,
        "Back 32 mN": 3,
        'Back LS': 4,
        "Back 256 mN": 5,
        "Eyes Open": 6,
        "Eyes Closed": 7,
}

### Compute functional connectivity for each subject in each group (surrogate data)

In [ ]:
all_sub_con_CP = []

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})"
    stc_path = stc_parent_path / f"/FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # Chronics (all) group
    if not load_group_data_flag:
        sub_con_CP = {}
        for sub_id in sub_ids_CP:
            #print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Chronic Pain",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                tmin,
                tmax,
                sfreq,
                left_pain_ids=None,
                right_pain_ids=None,
                bilateral_pain_ids=None,
                include_LS=False,
            )
            
            sub_con_CP[sub_id] = sub_avg_cons
        all_sub_con_CP.append(sub_con_CP)
    utils.clear_display()

In [33]:
all_sub_con_HC = []

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})"
    stc_path = stc_parent_path / f"/FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # Healthy Control group
    if not load_group_data_flag:
        sub_con_HC = {}
        for sub_id in sub_ids_HC:
            #print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Healthy Controls",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                tmin,
                tmax,
                sfreq,
                left_pain_ids=None,
                right_pain_ids=None,
                bilateral_pain_ids=None,
                include_LS=False,
            )
            sub_con_HC[sub_id] = sub_avg_cons
        all_sub_con_HC.append(sub_con_HC)
    utils.clear_display()

../../Data/Source Time Courses (MNE)/FFT Permuted Data 1
Reading stimulus labels and pain ratings for Subject C10...

*stim_labels length = 57*
*pain_ratings_raw length = 57*


Computing wpli2_debiased Hand 256 mN delta for C10...
+----------------+----------------+
| Subject        | C10            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be comput

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased WPLI Square
    computing cross-spectral density for epoch 1
    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C10...
+----------------+----------------+
| Subject        | C10            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10       

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C11...
+----------------+----------------+
| Subject        | C11            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 9              |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS win

...

    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C6....
+----------------+----------------+
| Subject        | C6.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased WPLI Square
    computing cross-spectral

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C7....
+----------------+----------------+
| Subject        | C7.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Us

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C9....
+----------------+----------------+
| Subject        | C9.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    comput

In [15]:
all_sub_con_WSP = []

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})"
    stc_path = stc_parent_path / f"/FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # CLBP with WSP
    if not load_group_data_flag:
        sub_con_WSP = {}
        for sub_id in sub_ids_WSP:
            print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Chronic Pain w/ WSP",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                tmin,
                tmax,
                sfreq,
                left_pain_ids=None,
                right_pain_ids=None,
                bilateral_pain_ids=None,
                include_LS=False,
            )
            sub_con_WSP[sub_id] = sub_avg_cons
        all_sub_con_WSP.append(sub_con_WSP)
    utils.clear_display()

../../Data/Source Time Courses (MNE)/FFT Permuted Data 1
018
Reading stimulus labels and pain ratings for Subject 018...

*stim_labels length = 73*
*pain_ratings_raw length = 73*


Computing wpli2_debiased Hand 256 mN delta for 018...
+----------------+----------------+
| Subject        | 018            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be co

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 018...
+----------------+----------------+
| Subject        | 018            |
+----------------+----------------+
| Condition      | 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 022...
+----------------+----------------+
| Subject        | 022            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+-----

...

    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 045...
+----------------+----------------+
| Subject        | 045            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 9
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 046...
+----------------+----------------+
| Subject        | 046            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 9              |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 052...
+----------------+----------------+
| Subject        | 052            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connecti

In [16]:
all_sub_con_LP = []

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})"
    stc_path = stc_parent_path / f"/FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # CLBP with LP
    if not load_group_data_flag:
        sub_con_LP = {}
        for sub_id in sub_ids_LP:
            print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Chronic Pain w/ Localized Pain",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                tmin,
                tmax,
                sfreq,
                left_pain_ids=None,
                right_pain_ids=None,
                bilateral_pain_ids=None,
                include_LS=False,
            )
            sub_con_LP[sub_id] = sub_avg_cons
        all_sub_con_LP.append(sub_con_LP)
    utils.clear_display()

../../Data/Source Time Courses (MNE)/FFT Permuted Data 1
020
Reading stimulus labels and pain ratings for Subject 020...

*stim_labels length = 80*
*pain_ratings_raw length = 80*


Computing wpli2_debiased Hand 256 mN delta for 020...
+----------------+----------------+
| Subject        | 020            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be co

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shap

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 021...
+----------------+----------------+
| Subject        | 021            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 17             |
+----------------+----------------+
| Band           | alpha          |
+----------------+----------------+
| Method         | wpli2_debiased |
+--------------

...

    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 049...
+----------------+----------------+
| Subject        | 049            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    fre

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 050...
+----------------+----------------+
| Subject        | 050            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+-----

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 056...
+----------------+----------------+
| Subject        | 056            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0

In [36]:
all_group_con_HC = []
all_group_con_CP = []
all_group_con_WSP = []
all_group_con_LP = []

for rep in range(0, repetitions):
    all_group_con_HC.append(fc.compute_group_con(all_sub_con_HC[rep], conditions, con_methods, band_names))
    all_group_con_CP.append(fc.compute_group_con(all_sub_con_CP[rep], conditions, con_methods, band_names))
    all_group_con_WSP.append(fc.compute_group_con(all_sub_con_WSP[rep], conditions, con_methods, band_names))
    all_group_con_LP.append(fc.compute_group_con(all_sub_con_LP[rep], conditions, con_methods, band_names))

Group connectivity completed.
Group connectivity completed.
Group connectivity completed.
Group connectivity completed.


In [18]:
# Highlight p-values with red box automatically
highlight_pvals=False

# Make any non-significant squares white
show_only_significant=True

# Set title automatically
set_title=True

# Show values in the matrix quadrants
show_fc_vals=True

# *KEEP FALSE* Round negative values to zero 
# (Vinck. et al 2011 has negative values for WPLI 2 Debiased. NO ROUNDING)
round_neg_vals=False

# Plot 3D brain visualization (for AEC only)
plot_brain=False

# Save tables to txt files
save_txt=False
# save_txt=True

### Collect p-values for surrogate data

In [37]:
surrogate_p_values = {}

#theta, alpha, beta, low-gamma, high-gamma
for repetition in range(0, repetitions):
    # CP Widespread Pain vs Healthy Control
    these_group_names = ["Chronic Pain", "Healthy Control","Mann_Whitney_U_test"]
    these_group_cons = [all_group_con_CP[repetition], all_group_con_HC[repetition]]
    these_sub_cons = [all_sub_con_CP[repetition], all_sub_con_HC[repetition]]
    these_sub_ids = [sub_ids_CP, sub_ids_HC]

    surrogate_p_values[repetition+1] = {}
    for condition in conditions:
        if 'Eyes' in condition:
            # change S1-lh and S1-rh in roi_acronyms to S1-i and S1-c
            roi_acronyms_resting = roi_acronyms.copy()
            roi_acronyms_resting[roi_acronyms.index("S1-lh")] = "S1-i"
            roi_acronyms_resting[roi_acronyms.index("S1-rh")] = "S1-c"
        
        surrogate_p_values[repetition+1][condition] = {}
        for method in con_methods:
            # Ignore some specific condition/method combinations
            if condition=="Hand 256 mN" and "aec" in method:
                continue
            elif condition=="Eyes Open" and method=="wpli2_debiased":
                continue
            
            surrogate_p_values[repetition+1][condition][method] = {}
            for band in band_names:   
                # For each of the top 3 connections, add the connection, its frequency, and its mean strength to a list  
                top_3_info=None
                group_con = None
                for group_con,group_name,sub_con in zip(these_group_cons, these_group_names, these_sub_cons):
                    table_data = []  
                    top_3_info = group_con[condition][method][band]["top 3"] 
                    #print(f"Top 3 Connections in {group_name} group")
                    for i in range(3):  
                        connection = top_3_info["connections"][i]  
                        frequency = top_3_info["frequency"][i]  
                        mean_strength = top_3_info["mean strength"][i]  
                        if 'Eyes' in condition:
                            table_data.append([f"{roi_acronyms_resting[connection[0]]} <-> {roi_acronyms_resting[connection[1]]}", f"{frequency}/{len(sub_con)}", mean_strength]) 
                        else:
                            table_data.append([f"{roi_acronyms[connection[0]]} <-> {roi_acronyms[connection[1]]}", f"{frequency}/{len(sub_con)}", mean_strength]) 
                    
                    # Define table headers  
                    headers = ['Connection', 'Frequency', 'Mean Strength']  
                    
                    # Print the table  
                    # print(tabulate(table_data, headers, tablefmt='pretty'),'\n')  
                
                group1_stack = these_group_cons[0][condition][method][band]["data"]
                group2_stack = these_group_cons[1][condition][method][band]["data"]

                nepochs_1 = these_group_cons[0][condition]['num_epochs']
                nepochs_2 = these_group_cons[1][condition]['num_epochs']
                
                # Compute the Mann-Whitney U test
                p_values, means_1, sem_1, means_2, sem_2 = fc.mann_whitney_test(
                    group1_stack,
                    group2_stack,
                    roi_acronyms=roi_acronyms if 'Eyes' not in condition else roi_acronyms_resting,
                    sub_ids1=these_sub_ids[0],
                    sub_ids2=these_sub_ids[1],
                    condition=condition,
                    bilateral_pain_ids=bilateral_pain_ids,
                    round_neg_vals=round_neg_vals,
                    method=method)
            
                # Save the p-values to a dictionary
                surrogate_p_values[repetition+1][condition][method][band] = p_values

with open(stc_path / 'surrogate_data.csv', 'w') as csvfile:
    outfile = csv.writer(csvfile, delimiter=',')
    for repetition in range(0, repetitions):
        for condition in conditions:
            for method in con_methods:
                if condition=="Hand 256 mN" and "aec" in method:
                    continue
                elif condition=="Eyes Open" and method=="wpli2_debiased":
                    continue
                for band in band_names:
                    if band=="theta" and condition=="Hand 256 mN":
                        headers = [repetition, condition, method, band]
                        outfile.writerow(headers)
                        outfile.writerows(surrogate_p_values[repetition+1][condition][method][band])
                        outfile.writerow('\n')

### Compute functional connectivity for each subject in each group (original data)

In [39]:
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

In [40]:
# All Chronic Pain
if not load_group_data_flag:
    sub_con_CP = {}
    for sub_id in sub_ids_CP:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Chronic Pain",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            tmin,
            tmax,
            sfreq,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_CP[sub_id] = sub_avg_cons            
        
    utils.clear_display()

018
Reading stimulus labels and pain ratings for Subject 018...

*stim_labels length = 73*
*pain_ratings_raw length = 73*

Left pain, -rh is already contralateral

Computing wpli2_debiased Hand 256 mN delta for 018...
+----------------+----------------+
| Subject        | 018            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 018...
+----------------+-----------

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 020...
+----------------+----------------+
| Subject        | 020            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+--------

...

    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 050...
+----------------+----------------+
| Subject        | 050            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Us

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 057...
+----------------+----------------+
| Subject        | 057            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 059...
+----------------+----------------+
| Subject        | 059            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 15             |
+----------------+----------------+
| Band           | alpha          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 8.8Hz..12.8Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased WPLI Square
    computing cross-spectral 

In [41]:
# All Healthy Controls
if not load_group_data_flag:
    sub_con_HC = {}
    for sub_id in sub_ids_HC:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Healthy Controls",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            tmin,
            tmax,
            sfreq,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_HC[sub_id] = sub_avg_cons            
        
    utils.clear_display()

C10
Reading stimulus labels and pain ratings for Subject C10...

*stim_labels length = 57*
*pain_ratings_raw length = 57*

Left pain, -rh is already contralateral

Computing wpli2_debiased Hand 256 mN delta for C10...
+----------------+----------------+
| Subject        | C10            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C10...
+----------------+----------------+
| Subject        | C10            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+-----

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 9
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C11...
+----------------+----------------+
| Subject        | C11            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 9              |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: 

...

    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C6....
+----------------+----------------+
| Subject        | C6.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C7....
+----------------+----------------+
| Subject        | C7.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS wi

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for C9....
+----------------+----------------+
| Subject        | C9.            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+-----

In [42]:
# CLBP with WSP
if not load_group_data_flag:
    sub_con_WSP = {}
    for sub_id in sub_ids_WSP:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Chronic Widespread Pain",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            tmin,
            tmax,
            sfreq,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_WSP[sub_id] = sub_avg_cons            
        
    utils.clear_display()

018
Reading stimulus labels and pain ratings for Subject 018...

*stim_labels length = 73*
*pain_ratings_raw length = 73*

Left pain, -rh is already contralateral

Computing wpli2_debiased Hand 256 mN delta for 018...
+----------------+----------------+
| Subject        | 018            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 022...
+----------------+----------------+
| Subject        | 022            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+-----

...

    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 045...
+----------------+----------------+
| Subject        | 045            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only usi

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 046...
+----------------+----------------+
| Subject        | 046            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 9              |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 13.6Hz..29.5Hz (21 points)
    connectivity scores will be averaged for each band
    Usi

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 052...
+----------------+----------------+
| Subject        | 052            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | beta           |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    comput

In [43]:
# CLBP with Localized Pain
if not load_group_data_flag:
    sub_con_LP = {}
    for sub_id in sub_ids_LP:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Chronic Pain",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            tmin,
            tmax,
            sfreq,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_LP[sub_id] = sub_avg_cons            
        
    utils.clear_display()

020
Reading stimulus labels and pain ratings for Subject 020...

*stim_labels length = 80*
*pain_ratings_raw length = 80*

Left pain, -rh is already contralateral

Computing wpli2_debiased Hand 256 mN delta for 020...
+----------------+----------------+
| Subject        | 020            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 20             |
+----------------+----------------+
| Band           | delta          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 0.0Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shap

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 021...
+----------------+----------------+
| Subject        | 021            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 17             |
+----------------+----------------+
| Band           | alpha          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
  

...

    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 049...
+----------------+----------------+
| Subject        | 049            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | alpha          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 8.8Hz..12.8Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: 

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN alpha for 050...
+----------------+----------------+
| Subject        | 050            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           | alpha          |
+----------------+----------------+
| Method         | wpli2_debiased |
+----------------+----------------+
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..1.250s for estimation (501 points)
    frequencies: 8.8Hz..12.8Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Debiased WPLI Square
    computing cross-spectral density for epoch 1
    computing c

/home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne_connectivity/spectral/epochs.py:186: RuntimeWarning: divide by zero encountered in scalar divide
  5.0 / np.min(fmin),
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:159: RuntimeWarning: fmin=0.000 Hz corresponds to 0.000 < 5 cycles based on the epoch length 1.252 sec, need at least inf sec epochs or fmin=3.992. Spectrum estimate will be unreliable.
  con_epochs = mne_conn.spectral_connectivity_epochs(


    the following metrics will be computed: Debiased WPLI Square
    computing cross-spectral density for epoch 1
    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    assembling connectivity matrix
[Connectivity computation done]
*con_epochs shape = (144, 1)*
*data shape = (12, 12)*

Computing wpli2_debiased Hand 256 mN beta for 056...
+----------------+----------------+
| Subject        | 056            |
+----------------+----------------+
| Condition      | Hand 256 mN    |
+----------------+----------------+
| Num. of epochs | 10             |
+----------------+----------------+
| Band           

### Collect p-values for original data

In [44]:
group_con_HC = fc.compute_group_con(sub_con_HC, conditions, con_methods, band_names)
group_con_CP = fc.compute_group_con(sub_con_CP, conditions, con_methods, band_names)
group_con_WSP = fc.compute_group_con(sub_con_WSP, conditions, con_methods, band_names)
group_con_LP = fc.compute_group_con(sub_con_LP, conditions, con_methods, band_names)

Group connectivity completed.
Group connectivity completed.
Group connectivity completed.
Group connectivity completed.


In [45]:
original_p_values = {}

##need to split by band condition etc cant just have one giant list ugh come back to this after lunch

# CP Widespread Pain vs Healthy Control
these_group_names = ["Chronic Pain", "Healthy Control","Mann_Whitney_U_test"]
these_group_cons = [group_con_CP, group_con_HC]
these_sub_cons = [sub_con_CP, sub_con_HC]

for condition in conditions:
    # initialize dictionary
    original_p_values[condition] = {}
    
    if 'Eyes' in condition:
        # change S1-lh and S1-rh in roi_acronyms to S1-i and S1-c
        roi_acronyms_resting = roi_acronyms.copy()
        roi_acronyms_resting[roi_acronyms.index("S1-lh")] = "S1-i"
        roi_acronyms_resting[roi_acronyms.index("S1-rh")] = "S1-c"

    for method in con_methods:
        original_p_values[condition][method] = {}
        
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue

        for band in band_names:                   
            # Select groups for mann-whitney
            # For each of the top 3 connections, add the connection, its frequency, and its mean strength to a list  
            for group_con,group_name,sub_con in zip(these_group_cons, these_group_names, these_sub_cons):
                table_data = []  
                top_3_info = group_con[condition][method][band]["top 3"] 
                #print(f"Top 3 Connections in {group_name} group")
                for i in range(3):  
                    connection = top_3_info["connections"][i]  
                    frequency = top_3_info["frequency"][i]  
                    mean_strength = top_3_info["mean strength"][i]  
                    table_data.append([f"{roi_acronyms[connection[0]]} <-> {roi_acronyms[connection[1]]}", f"{frequency}/{len(sub_con)}", mean_strength]) 
                
                # Define table headers  
                headers = ['Connection', 'Frequency', 'Mean Strength']  

            group1_stack = these_group_cons[0][condition][method][band]["data"]
            group2_stack = these_group_cons[1][condition][method][band]["data"]

            nepochs_1 = these_group_cons[0][condition]['num_epochs']
            nepochs_2 = these_group_cons[1][condition]['num_epochs']

            # Compute the Mann-Whitney U test
            p_values, means_1, sem_1, means_2, sem_2 = fc.mann_whitney_test(
                group1_stack,
                group2_stack,
                roi_acronyms=roi_acronyms if 'Eyes' not in condition else roi_acronyms_resting,
                sub_ids1=these_sub_ids[0],
                sub_ids2=these_sub_ids[1],
                condition=condition,
                bilateral_pain_ids=bilateral_pain_ids,
                round_neg_vals=round_neg_vals,
                method=method)
            
            # Save the p-values to a dictionary
            original_p_values[condition][method][band] = p_values


with open(stc_path / 'original_data.csv', 'w') as csvfile:
    outfile = csv.writer(csvfile, delimiter=',')
    for condition in conditions:
        for method in con_methods:
            if condition=="Hand 256 mN" and "aec" in method:
                continue
            elif condition=="Eyes Open" and method=="wpli2_debiased":
                continue
            for band in band_names:
                outfile.writerow(condition + ' ' + method + ' ' + band)
                outfile.writerows(original_p_values[condition][method][band])
            

### Calculate new permutation p-value based on num. p-values greater than orig. p-value

In [46]:
all_ps = {}

for condition in conditions:
    all_ps[condition] = {}
    for method in con_methods:
        all_ps[condition][method] = {}
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue

        for band in band_names:
            new_p_values = [[0]*len(roi_names)]*len(roi_names)
            for region1 in range(0, len(roi_names)):
                for region2 in range(0, len(roi_names)):
                    if region2 > region1:
                        original_p_value = original_p_values[condition][method][band][region1][region2]
                        pos_vals = []
                        neg_vals = []
                        new_p_value = 0
                        for rep in range(0, repetitions):
                            surrogate_p_value = surrogate_p_values[rep+1][condition][method][band][region1][region2]
                            temp_p_value = surrogate_p_value - original_p_value
                            if temp_p_value < 0:
                                neg_vals.append(temp_p_value)
                            elif temp_p_value > 0:
                                pos_vals.append(temp_p_value)

                        new_p_value = len(pos_vals)/repetitions
                        print(len(pos_vals))
                        new_p_values[region1][region2] = new_p_value
                    #else:
                        #new_p_values[region1][region2] = None

            # Save the p-values to a dictionary
            all_ps[condition][method][band] = new_p_values
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
1
0
0
1
0
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
1
0
1
0
0
1
0
0
1
0
0
1
1
0
0
0
1
1
0
1
1
1
0
1
1
0
1
1
0
0
1
1
0
1
1
1
0
1
1
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
1


In [47]:
# Initialize a dictionary to hold the comparison results
comparison_results = {}

# Iterate over conditions
for condition in conditions:
    comparison_results[condition] = {}
    
    # Iterate over methods
    for method in con_methods:
        comparison_results[condition][method] = {}
        
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue
        # Iterate over bands
        for band in band_names:
            permutation_matrix = [[0 for _ in range(len(roi_names))] for _ in range(len(roi_names))]
            # Compare each p-value in the surrogate matrix with the corresponding original p-value
            for repetition in range(0, repetitions):
                for i in range(0, len(roi_acronyms)):
                    for j in range(0, i):
                        if surrogate_p_values[repetition+1][condition][method][band][i][j] <= original_p_values[condition][method][band][i][j]:
                            permutation_matrix[i][j] = permutation_matrix[i][j] + 1
                        if repetition == repetitions-1:
                            permutation_matrix[i][j] = permutation_matrix[i][j]/repetitions
            # Store the results for the current condition, method, and band
            comparison_results[condition][method][band] = permutation_matrix

with open(stc_path / 'comparison_data.csv', 'w') as csvfile:
    outfile = csv.writer(csvfile, delimiter=',')
    for condition in conditions:
        for method in con_methods:
            if condition=="Hand 256 mN" and "aec" in method:
                continue
            elif condition=="Eyes Open" and method=="wpli2_debiased":
                continue
            for band in band_names:
                headers = [condition, method, band]
                outfile.writerow(headers)
                outfile.writerows(comparison_results[condition][method][band])
                outfile.writerow('\n')
